# Automatically find peaks in the spectrum

## Imports

Standard library imports

In [ ]:
from pathlib import Path
from typing import Union

Third-party  imports

In [ ]:
import numpy as np
import scipy.signal as si
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator
from astropy.table import Table
from astropy.wcs import WCS
from astropy.io import fits
import seaborn as sns
import cmasher as cmr
from mpdaf.obj import Cube, Image, Spectrum
import regions as rg

sns.set_context("talk")

Will's libraries

In [ ]:
import wcsfile

## Paths for data files

Files in `small_data_path` should not be too large to be checked into git

In [ ]:
data_path = Path.cwd().parent.parent / "big-data" / "ngc346new"
small_data_path = Path.cwd().parent.parent / "data"

## Read in data files

In [ ]:
cube = Cube(str(data_path / "n346-muse-csub-101.fits"))

Switch to reading the fits files directly

In [ ]:
star_mask = fits.open(small_data_path / "n346-mask-stars.fits")[1].data
yso_mask = fits.open(small_data_path / "n346-mask-yso.fits")[1].data

In [ ]:
Image(data=star_mask - yso_mask).plot()

In [ ]:
mask = np.where(yso_mask > 0.0, 1.0, np.nan)
spec_yso = np.nanmean(cube.data * mask[None, :, :], axis=(1, 2))

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
Spectrum(data=spec_yso, wave=cube.wave).plot()
ax.set_ylim(-50, 200)
ax.set_xlim(8100, 8600)
...;

In [ ]:
mask = np.where(yso_mask + star_mask == 0.0, 1.0, np.nan)
spec_all = np.nanmean(cube.data * mask[None, :, :], axis=(1, 2))

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
Spectrum(data=spec_all, wave=cube.wave).plot()
ax.set_ylim(-5, 20)
ax.set_xlim(6200, 6800)
...;

In [ ]:
peaks, props = si.find_peaks(spec_yso, prominence=5.0,   distance=4)

In [ ]:
peaks

In [ ]:
props['prominences']

In [ ]:
wave = cube.wave

In [ ]:
wave.coord()[peaks]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
Spectrum(data=spec_yso, wave=cube.wave).plot()
ymax=200
ax.scatter(wave.coord()[peaks], np.minimum(ymax, props["prominences"]), marker=".", color="r")
ax.set_ylim(-50, ymax * 1.1)
ax.set_xlim(8100, 8600)
...;

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
Spectrum(data=spec_yso, wave=cube.wave).plot()
ymax=200
ax.scatter(wave.coord()[peaks], np.minimum(ymax, props["prominences"]), marker=".", color="r")
ax.set_ylim(-50, ymax * 1.1)
ax.set_xlim(6200, 6800)
...;

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
Spectrum(data=spec_yso, wave=cube.wave).plot()
ymax=200
ax.scatter(wave.coord()[peaks], np.minimum(ymax, props["prominences"]), marker=".", color="r")
ax.set_ylim(-50, ymax * 1.1)
ax.set_xlim(4600, 5400)
...;

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
Spectrum(data=spec_yso, wave=cube.wave).plot()
ymax=200
ax.scatter(wave.coord()[peaks], np.minimum(ymax, props["prominences"]), marker=".", color="r")
ax.set_ylim(-0.1 * ymax, ymax * 1.1)
ax.set_xlim(6950, 7300)
...;

In [ ]:
peaks_all, props_all = si.find_peaks(spec_all, prominence=0.3, distance=4)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
Spectrum(data=spec_all, wave=cube.wave).plot()
ymax=5
ax.scatter(wave.coord()[peaks_all], np.minimum(ymax, props_all["prominences"]), marker="x", color="m")
ax.set_ylim(-0.1 * ymax, ymax * 1.1)
ax.set_xlim(6950, 7300)
...;

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
Spectrum(data=spec_all, wave=cube.wave).plot()
ymax=10
ax.scatter(wave.coord()[peaks_all], np.minimum(ymax, props_all["prominences"]), marker="x", color="m")
ax.set_ylim(-0.1 * ymax, ymax * 1.1)
ax.set_xlim(4600, 5400)
...;

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
Spectrum(data=spec_all, wave=cube.wave).plot()
ymax=20
ax.scatter(wave.coord()[peaks_all], np.minimum(ymax, props_all["prominences"]), marker="x", color="m")
ax.set_ylim(-0.1 * ymax, ymax * 1.1)
ax.set_xlim(8100, 8600)
...;

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
Spectrum(data=spec_all, wave=cube.wave).plot()
ymax=20
ax.scatter(wave.coord()[peaks_all], np.minimum(ymax, props_all["prominences"]), marker="x", color="m")
ax.set_ylim(-0.1 * ymax, ymax * 1.1)
ax.set_xlim(8600, 9300)
...;

In [ ]:
props_all['prominences']

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(props_all, index=wave.coord()[peaks_all])
df